<a href="https://colab.research.google.com/github/MelDashti/Smart-Chatbot/blob/master/AIChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning into 'Smart-Chatbot'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 42 (delta 10), reused 17 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 958.19 KiB | 3.54 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [1]:
import os
os.chdir('/content/Smart-Chatbot') # Here we set the working directory

import pandas as pd
df = pd.read_csv("dataset.csv")  # CSV

import pandas as pd
df = pd.read_json("dataset.json")  # JSON

with open("dataset.txt", "r") as f: # TXT
    data = f.readlines()


FileNotFoundError: [Errno 2] No such file or directory: '/content/Smart-Chatbot'

In [1]:
# Install necessary libraries
!pip install trl
!pip install unsloth

# Standard library imports
import os
import warnings

# Third-party library imports
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer, AutoTokenizer, AutoModelForSequenceClassification
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import Dataset

# Configure warnings and matplotlib
warnings.filterwarnings("ignore")
%matplotlib inline
plt.style.use('ggplot')

# Set device (GPU if available, otherwise CPU)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
cuda:0


In [7]:
# WE are using lama with 1B parameters.


max_seq_length = 1024  # imo its enough for a simple AI chatbot
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True, # enables efficient 4-bit quantization
    dtype=None,
)
# we use parameter efficient fine tuning like we learnt in LLM which applied LORA techniques. This approach
# focuses on fine tuning only specific layers or parts of the model, rather than the entire network.
# r = 16 and lora_alpha = 16 adjusts the complexity and scaling of these adaptations.
#  target modules specifies which layers of the model should be adapted, which include key components involed
# in attention mechanisms like q_proj and k_proj and v_proj.
# use_rslora activates Rank stabalized LORA, which improves the stability of the fine tuning process.
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    # q_proj, k_proj, v_proj: Handle the query, key, and value projections in the attention mechanism, essential for capturing contextual information.
    # up_proj, down_proj: Layers in feedforward networks. o_proj: Combines attention heads’ output. gate_proj: Controls flow in certain feedforward networks.
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state = 32,
    loftq_config = None,
)
print(model.print_trainable_parameters())

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.11.7 patched 16 layers with 16 QKV layers, 16 O layers and 0 MLP layers.


trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750
None


Sample Data For The Arol Group Chatbot.
This part will be replaced after we data scrap the websites later.

In [8]:
sample_data = [
    {
        "Query": "What are the different types of bottle caps AROL Group manufactures?",
        "Response": "AROL Group manufactures various types of bottle caps, including screw caps, crown caps, and corks, designed for different industries and bottle types."
    },
    {
        "Query": "Can you explain how the capping machines work?",
        "Response": "Capping machines from AROL use precision automation to apply caps on bottles at high speed, ensuring a tight and consistent seal to prevent leakage and maintain product quality."
    },
    {
        "Query": "What industries does AROL Group provide solutions for?",
        "Response": "AROL Group provides capping and sealing solutions for industries such as food & beverage, personal care, pharmaceuticals, and chemical products."
    },
    {
        "Query": "How do AROL's machines ensure product safety?",
        "Response": "AROL machines use advanced inspection systems to detect any defects in capping, ensuring product safety by maintaining a consistent and contamination-free seal."
    },
    {
        "Query": "Does AROL offer maintenance support for their capping machines?",
        "Response": "Yes, AROL offers comprehensive maintenance support, including regular inspections, troubleshooting, and part replacements to keep machines running efficiently."
    },
    {
        "Query": "What are the environmental impacts of AROL's products?",
        "Response": "AROL is committed to sustainability, designing energy-efficient machines and encouraging the use of recyclable caps to minimize environmental impact."
    },
    {
        "Query": "Are AROL's machines customizable for different bottle sizes?",
        "Response": "Yes, AROL's capping machines are highly customizable to fit a wide range of bottle sizes, cap types, and production line configurations."
    },
    {
        "Query": "How does AROL ensure precision in high-speed bottling?",
        "Response": "AROL ensures precision with advanced robotics and servo-controlled systems that maintain accuracy and consistency, even at high production speeds."
    },
    {
        "Query": "What quality standards does AROL adhere to?",
        "Response": "AROL adheres to strict international quality standards, including ISO certifications, to ensure the reliability and safety of its products."
    },
    {
        "Query": "Does AROL offer training for operating their capping machines?",
        "Response": "Yes, AROL offers training programs to help operators understand and efficiently use their capping machines, covering safety, maintenance, and operational techniques."
    }
]


Now we convert the sample data to prompt template that we want to use for our chatbot.

In [9]:
import pandas as pd

# first we convert sample data to a DataFrame
df = pd.DataFrame(sample_data)

data_prompt = """
You are a customer support assistant for AROL Group, specialized in bottle caps and capping technologies.
Your goal is to provide accurate, clear, and helpful responses to queries regarding our products, processes, and technologies.

### Input:
{}

### Instructions:
- Provide a concise and informative response regarding bottle cap manufacturing or capping technology.
- If there are any technical details or product features mentioned, explain them in simple terms.
- If the query highlights any concerns, provide recommendations or solutions.
- Ensure the response is relevant to the specific question asked.

### Response:
{}
"""

EOS_TOKEN = "</s>"

def formatting_prompt(df):
    inputs = df["Query"]
    outputs = df["Response"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}


formatted_data = formatting_prompt(df) # Convert formatted data into the form the model expects
print(formatted_data["text"][1])
# we use templates so that we can fine tune our model better with instructions for how to analyze the input
# data and generate the output data.




You are a customer support assistant for AROL Group, specialized in bottle caps and capping technologies.
Your goal is to provide accurate, clear, and helpful responses to queries regarding our products, processes, and technologies.

### Input:
Can you explain how the capping machines work?

### Instructions:
- Provide a concise and informative response regarding bottle cap manufacturing or capping technology.
- If there are any technical details or product features mentioned, explain them in simple terms.
- If the query highlights any concerns, provide recommendations or solutions.
- Ensure the response is relevant to the specific question asked.

### Response:
Capping machines from AROL use precision automation to apply caps on bottles at high speed, ensuring a tight and consistent seal to prevent leakage and maintain product quality.
</s>


In [10]:
training_data = Dataset.from_pandas(df) # here we convert pandas dataframe into a hugging face dataset object
training_data = training_data.map(formatting_prompt, batched=True) # Here we apply the formatting func to each element of the dataset using map method.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Training the Model with the Trainer API
Goal: Use the Trainer API to actually fine-tune the model on the formatted dataset. This step leverages all previous configurations for efficient training.

Process:

The formatted data is fed into the Trainer as input for model training.
The Trainer uses LoRA fine-tuning to adjust only specific layers, optimizing performance while keeping memory usage low.
Purpose: This final step leverages all previous configurations and formatted data to train the model. The Trainer applies gradient updates to the specified layers according to LoRA parameters, optimizing the model for the task without requiring massive resources.

In [11]:

trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=training_data,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=40,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1 | Num Epochs = 40
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 40
 "-____-"     Number of trainable parameters = 3,407,872
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: meldashti (meldashti-politecnico-di-torino). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,0.790500
2,0.790500
3,0.775800
4,0.743100
5,0.693300
6,0.629400
7,0.554000
8,0.469000
9,0.376400
10,0.284200


TrainOutput(global_step=40, training_loss=0.17865518364124, metrics={'train_runtime': 31.7162, 'train_samples_per_second': 1.261, 'train_steps_per_second': 1.261, 'total_flos': 239997907107840.0, 'train_loss': 0.17865518364124, 'epoch': 40.0})

## Inference Mode: Applying Knowledge to User Queries

Now that the model is trained, it's ready to assist users with their inquiries about AROL Group products and services. In this phase, the model leverages the knowledge gained during fine-tuning to generate informative and helpful responses.

### User Interaction:

Users will input their questions or requests related to AROL Group's offerings, such as:

*   "What types of bottle caps are suitable for carbonated drinks?"
*   "How do I maintain my AROL capping machine for optimal performance?"
*   "Can AROL's solutions be customized for my specific production needs?"

### Model Response:

The model processes the user's input and generates a response based on the information it has learned. These responses will be:

*   **Tailored to AROL Group's domain:** The model's knowledge is focused on bottle caps, capping technologies, and related services offered by AROL Group.
*   **Informative and accurate:** The responses aim to provide clear and relevant answers to user queries, leveraging the data it was trained on.

In [28]:
text = "What does AROL Group manufacture?"
model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 100, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]
print("Answer of the question is:", answer)


Answer of the question is: 

AROL Group manufactures bottle caps and capping technologies. Bottle caps are used to seal bottles and jars, while capping technologies are used to seal bottles and jars with a lid. AROL Group offers a wide range of bottle caps and capping technologies, including metal, plastic, and glass bottle caps and capping technologies. AROL Group's bottle caps and capping technologies are designed to meet the needs of various industries, including food and beverage, pharmaceutical, and personal care. AROL Group's
